# Determine Phase Connectivity



This notebook consists of cells that can be used to compute the connectivity of a phase in the through-plane direction, grouping into different types of connection including:

- full: fully connected from the frist slice to the last
- top: connected only to the first slice
- botton: connected only to the last slice
- not: not connected, that is, isolated features of the phase

Note that the input image should be such that the through-plane slices progress from closer to the CL to the flow channel

**Usage**: In the file menu, click on *Run>Run All Cells*. Then, enter input as prompted
  
**output**: 

    - a csv file containing computed proportion of each category of connection is stored in the image folder povided
    - an image stack file with the phase segmented into the different connection types



- author: Akin Aroge
- email: arogeakindele@gmail.com
- version: 0.2
- created: 26-04-2022
- updated: 24-08-2022

In [ ]:
import numpy as np
import os
from skimage.io import imread, imsave
import imaging as ig
import pandas as pd
import skimage as sk
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = 8, 6

f_dir = str(input('Enter the containing folder of the image (full path):'))
fname = str(input("Enter the file name (including extension, e.g 'image.tiff'):"))
phase_val = int(input('Enter the value of the water phase pixels: '))
pixel_size = float((input('Enter pixel size of the image:')))

# === for testing using sample image ====
# f_dir = r'..\sample-files'
# fname = r'small-sample-rois-1.tif'
# phase_val = 2
# pixel_size = 1.54


im_path = os.path.join(f_dir, fname)
im = imread(im_path)

im_conn_label, im_span_label, comp_span_dict, connectivity_key = ig.analysis.get_connectivity(im, phase_val=phase_val, slack=3)

## ====== discrete compoent labels as fraction of total water ====== ##
conn_type_num, count = np.unique(im_conn_label, return_counts=True)

zero_idx = np.where(conn_type_num==0)

# remove zero, it is air
conn_type_num = np.delete(conn_type_num, zero_idx)
counts = np.delete(count, zero_idx)
sum_counts = counts.sum()

conn_type = [connectivity_key[num] for num in conn_type_num]

fig, ax = plt.subplots(figsize=(6,6))

fracs = counts/sum_counts
bars = ax.bar(x=conn_type, height=fracs)

ax.set_ylabel('liquid water fraction')
ax.set_xlabel('Through-plane water connectivity')
#ax.set_yscale('log')
plt.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

data = pd.DataFrame({'connectivity':conn_type,
            'fraction': fracs})

result_fname = 'connectivity-proportion.csv'
data.to_csv(os.path.join(f_dir, result_fname), index=False)

imsave(os.path.join(f_dir, 'connectivity-image.tif'), im_conn_label)

## ====== portion of total amount of water ====== ##
im_water_span = im_span_label[im_span_label > 0]
full_span = im.shape[0]

fig, ax = plt.subplots()

weights = np.ones(len(im_water_span))/ len(im_water_span)
ax.hist((im_water_span.flatten())*pixel_size, bins=30, weights=weights )
#ax.set_yscale('log')

ax.set_xlabel('TP span ($\mu m$)')
ax.set_ylabel('portion of total amount of water')
# - 0.5 is
ax.axvline(x=(full_span)*pixel_size, ls='--', c='r', label='GDL TP length')

ax.legend()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)